# A1111 Model Installer — SDXL (1.0)

Downloads **SDXL** assets for Automatic1111 with visible **progress bars**.

- **Data root (WEBUI_ROOT):** `/workspace/a1111`
- **Models folder:** `/workspace/a1111/models/...`

### What’s covered
- **Checkpoints** (SDXL base/refiner)
- **VAEs** (SDXL)
- **ControlNet XL** models
- **Optional extras** (AnimateDiff XL, etc.)

**Re-run safe**: existing files are skipped.

> Many SDXL weights require accepting a license on Hugging Face. Set your `HF_TOKEN` in the RunPod template.

In [ ]:
# Install helpers once if needed (idempotent)
# !pip install -q tqdm requests huggingface_hub
print("Downloader ready for SDXL.")

In [ ]:
import os
from pathlib import Path

# A1111 data root (matches WEBUI_ROOT)
DATA_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))

# Standard subfolders used by A1111
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"
EXTRA_DIR= DATA_ROOT / "models/Extras"

for d in (CKPT_DIR, VAE_DIR, CN_DIR, EXTRA_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet XL:", CN_DIR)
print("↳ Extras:", EXTRA_DIR)

## Progress-enabled download helpers
Supports both **Hugging Face** (`repo_id` + `filename`) and **direct URLs**.

In [ ]:
import os, shutil, requests, shlex, subprocess, time
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from urllib.parse import urlparse
from pathlib import Path

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024:
            return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
    tmp.replace(dest)
    print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")
    return dest

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None, token: str|None=None):
    dest = dest_dir / (dest_name or filename)
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=token or os.environ.get("HF_TOKEN"),
        local_dir=dest_dir,
        local_dir_use_symlinks=False,
        resume_download=True,
    )
    if Path(path) != dest:
        shutil.copy2(path, dest)
    print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")
    return dest

print("Helpers loaded.")

## Choose what to download (SDXL)

Uncomment or add items. **HF items** use `repo` + `filename`; **direct items** use `url`.

**Tip:** SDXL base/refiner usually require accepting a license and using `HF_TOKEN`.

In [ ]:
# --- SDXL checkpoints ---
CHECKPOINTS = [
    # Base & Refiner (commented by default – require TOS acceptance on HF)
    # {"kind": "hf", "repo": "stabilityai/stable-diffusion-xl-base-1.0",    "filename": "sd_xl_base_1.0.safetensors",   "dest": CKPT_DIR / "sd_xl_base_1.0.safetensors"},
    # {"kind": "hf", "repo": "stabilityai/stable-diffusion-xl-refiner-1.0", "filename": "sd_xl_refiner_1.0.safetensors","dest": CKPT_DIR / "sd_xl_refiner_1.0.safetensors"},

    # Add your favorite SDXL .safetensors (HF or direct)
    # {"kind": "direct", "url": "https://your.cdn/sdxl_favorite.safetensors", "dest": CKPT_DIR / "sdxl_favorite.safetensors"},
]

# --- SDXL VAEs ---
VAE_FILES = [
    # Popular SDXL VAE fix (public)
    {"kind": "hf", "repo": "madebyollin/sdxl-vae-fp16-fix", "filename": "sdxl_vae.safetensors", "dest": VAE_DIR / "sdxl_vae.safetensors"},
]

# --- ControlNet XL ---
# Examples: one from diffusers and one from lllyasviel (filenames saved friendly for A1111)
CONTROLNET_XL = [
    # diffusers/controlnet-sdxl-1.0 ships weights as diffusion_pytorch_model.safetensors
    # {"repo": "diffusers/controlnet-sdxl-1.0", "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "controlnet-sdxl-1.0.safetensors"},
    # lllyasviel XL examples (adjust names as needed)
    # {"repo": "lllyasviel/controlnet-sdxl-1.0", "filename": "controlnet-canny-sdxl-1.0.safetensors",   "save_as": CN_DIR / "controlnet-canny-sdxl-1.0.safetensors"},
    # {"repo": "lllyasviel/controlnet-sdxl-1.0", "filename": "controlnet-depth-sdxl-1.0.safetensors",   "save_as": CN_DIR / "controlnet-depth-sdxl-1.0.safetensors"},
]

# --- Optional extras (e.g., AnimateDiff XL motion module) ---
EXTRA_FILES = [
    # {"kind": "hf", "repo": "guoyww/animatediff", "filename": "SDXL/sdxl_motion_module.safetensors", "dest": EXTRA_DIR / "sdxl_motion_module.safetensors"},
]

print(f"Planned: {len(CHECKPOINTS)} ckpts, {len(VAE_FILES)} VAE, {len(CONTROLNET_XL)} CN-XL, {len(EXTRA_FILES)} extras")

## Run downloads
Edit the lists above, then run this cell. Large files show progress bars.

In [ ]:
errors = []

def run_items(items):
    for it in items:
        try:
            if it.get("kind") == "hf":
                download_hf(it["repo"], it["filename"], it["dest"].parent, dest_name=it["dest"].name)
            elif it.get("kind") == "direct":
                download_direct(it["url"], it["dest"])
            else:
                # ControlNet-style entries use repo/filename/save_as without "kind"
                if "repo" in it and "filename" in it and "save_as" in it:
                    download_hf(it["repo"], it["filename"], it["save_as"].parent, dest_name=it["save_as"].name)
        except Exception as e:
            errors.append((str(it), str(e)))

print("\n=== SDXL Checkpoints ===")
run_items(CHECKPOINTS)

print("\n=== SDXL VAEs ===")
run_items(VAE_FILES)

print("\n=== ControlNet XL ===")
run_items(CONTROLNET_XL)

print("\n=== Extras ===")
run_items(EXTRA_FILES)

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024:
            return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

print("\n=== Summary ===")
for p in sorted(list(CKPT_DIR.glob('*')) + list(VAE_DIR.glob('*')) + list(CN_DIR.glob('*')) + list(EXTRA_DIR.glob('*'))):
    try:
        print(f"✔ {p.name:55s} {human(p.stat().st_size)}")
    except Exception:
        pass
if errors:
    print("\nErrors:")
    for item, msg in errors:
        print("✖", item, "→", msg)
print("\nAll done.")

## (Optional) Restart the WebUI service
If you added new models while A1111 was running, restart it so the UI refreshes lists.

In [ ]:
print("Restarting A1111 via supervisord …")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True, timeout=20)
    if out.returncode == 0:
        print("✔ Restarted. Wait ~10–15s, then refresh the WebUI (7860).")
        print(out.stdout.strip())
    else:
        print("Supervisorctl failed; trying process restart…")
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI killed; supervisord will auto-restart. Refresh in ~10–15s.")
except Exception as e:
    print("Restart attempt hit an error:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")